# 0. Imports

In [10]:
from h2ogpte import H2OGPTE
import numpy as np
import pandas as pd
import os

# 1. Data Sets

In [4]:
songs = pd.read_csv('../data/songs_short.csv')
user = pd.read_csv('../data/user.csv')
history = pd.read_csv('../data/listening history.csv')

In [ ]:
print('Songs:')
print(songs.head())

print('Users:')
print(user.head())

print('User listening history:')
print(history.head())

# 2. h2o Model

In [9]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-e4vXVj35STQ15msFSdaeZPDo0xvCElTK4Q0hw9F1LUjgTJov',
)

In [11]:
# Create Collection
collection_id = client.create_collection(
    name="MusicRecco",
    description="Music Reccommender",
)

In [18]:
# Convert Data to Readable Format
songs_txt = songs.to_csv(index=False)
user_txt = user.to_csv(index=False)
history_txt = history.to_csv(index=False)

In [19]:
# Upload
songs_data = client.upload('songs_short.txt', songs_txt.encode())
user_data = client.upload('user.txt', user_txt.encode())
history_data = client.upload('listening_history.txt', history_txt.encode())

# Ingest the uploaded data
client.ingest_uploads(collection_id, [songs_data, user_data, history_data])

Job(id='e0c202c7-a81a-45a1-adb9-a53adffef26b', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 31, 6, 47, 42, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='f6f52470dca241bbb96b6ff5d2649e4a', status='Indexing done.'), JobStatus(id='36fea923bf79437c86bd632a60267d92', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 31, 6, 47, 56, tzinfo=TzInfo(UTC)), duration='14s', duration_seconds=14.0)

# 3. ChatBot Page

In [20]:
#Initiate Chat
chat_session_id = client.create_chat_session(collection_id)

In [21]:
with client.connect(chat_session_id) as session:
    # Simple Question for Document Collection
        answer = session.query(
        message="I am user sibyl. Reccomend me songs similar to my taste",
        system_prompt = 'Assume music and song to be the same word. When a question as for a similar music, recommend less than 5 music unless told otherwise. Find similar music based on genre and other factors such as danceability, loudness, speechiness and more. Just return the song name, unless stated otherwise.' ,
        rag_config={
            "rag_type": "rag",
        },
        ).content
        print(answer)


Based on your user information, you have listed rock, r&b, and pop as your preferred genres. Here are five songs that are similar to your taste:

1. "Bohemian Rhapsody" by Queen (rock)
2. "Crazy in Love" by Beyoncé ft. Jay-Z (r&b/pop)
3. "Mr. Brightside" by The Killers (rock/pop)
4. "No Scrubs" by TLC (r&b)
5. "I Wanna Dance with Somebody" by Whitney Houston (pop)
